#### Define a Bed reader and read a few values from it (they are a 2D numpy array)

In [2]:
import os
import numpy as np

from pysnptools.snpreader import Bed
snpreader = Bed("all.bed", count_A1=True)

# What is snpreader?
print(snpreader)
snpreader[:2,:4].read().val

Bed('all.bed',count_A1=True)


array([[0., 0., 1., 2.],
       [0., 0., 1., 1.]])

#### Convert a Bed reader into a distribution reader

In [3]:
from pysnptools.distreader import Snp2Dist
distreader = Snp2Dist(snpreader)
distreader

Snp2Dist(Bed('all.bed',count_A1=True))

#### Read a few values from the distribution reader. They are a 3D numpy array.

In [4]:
distdata = distreader[:2,:4].read()
distdata.val

array([[[1., 0., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]],

       [[1., 0., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 1., 0.]]])

#### Write the distribution values to a file.

In [5]:
from pysnptools.distreader import DistMemMap
DistMemMap.write('sample.dist.memmap',distdata)

DistMemMap('sample.dist.memmap')

##### Read from the file. Listing the individual ID's the SNP id's and the values.

In [6]:
distmemmap = DistMemMap('sample.dist.memmap')
print(distmemmap.iid)
print(distmemmap.sid)
print(distmemmap.val)

[['cid0P0' 'cid0P0']
 ['cid1P0' 'cid1P0']]
['snp625_m0_.03m1_.07' 'snp1750_m0_.02m1_.04' 'snp0_m0_.37m1_.24'
 'snp375_m0_.52m1_.68']
[[[1. 0. 0.]
  [1. 0. 0.]
  [0. 1. 0.]
  [0. 0. 1.]]

 [[1. 0. 0.]
  [1. 0. 0.]
  [0. 1. 0.]
  [0. 1. 0.]]]


#### Create an in-memory distribution
* Create via random
* Create a second distribution with the individuals reversed and with every 10th SNP.
* Print the start of the 2nd distribution's individual ID's the SNP id's and the values

In [7]:
from pysnptools.distreader import DistData
import numpy as np

np.random.seed(0)
iid_count = 100
sid_count = 1000
val = np.random.random((iid_count,sid_count,3))
val /= val.sum(axis=2,keepdims=True)  #make probabilities sum to 1
distdata = DistData(val=val,
                    iid=[('fam0','iid{0}'.format(i)) for i in range(iid_count)],
                    sid=['sid{0}'.format(s) for s in range(sid_count)]
                    )
distdata2 = distdata[::-1,::10].read()
print(distdata2.iid[:3])
print(distdata2.sid[:3])
print(distdata2.val[:3,:3])

[['fam0' 'iid99']
 ['fam0' 'iid98']
 ['fam0' 'iid97']]
['sid0' 'sid10' 'sid20']
[[[0.20081495 0.4737261  0.32545895]
  [0.52343568 0.16311684 0.31344748]
  [0.26296867 0.5548358  0.18219553]]

 [[0.29093073 0.36418333 0.34488594]
  [0.12887058 0.64865356 0.22247586]
  [0.00325359 0.69403215 0.30271426]]

 [[0.02445881 0.66399687 0.31154432]
  [0.33538909 0.35483828 0.30977264]
  [0.40125371 0.25403278 0.34471351]]]


In [8]:
# test bgen reading
from pysnptools.distreader.bgen import Bgen
bgenfile = r'D:\OneDrive\programs\pysnptools\pysnptools\examples\example.bgen'
bgen = Bgen(bgenfile) #!!!cmk make relative
bgen.shape

cmk


(500, 199)

In [9]:
#!!!cmk would be nice to have *MemMap writes know how to work from *Readers and have optional runners
memmapfile = 'example32.dist.memmap'
memmap = DistMemMap.write(memmapfile,bgen.read(dtype=np.float32))

In [10]:
bgensize = os.stat(bgenfile).st_size
memmapsize = os.stat(memmapfile).st_size
bgensize,memmapsize,memmapsize/bgensize

(665108, 1223090, 1.8389344286942872)

In [44]:
memmap.val[:5,:5,:]

memmap([[[           nan,            nan,            nan],
         [5.06591937e-03, 2.07519974e-03, 9.92858887e-01],
         [9.92095947e-01, 3.05176014e-04, 7.59887975e-03],
         [4.88280971e-03, 2.83812974e-02, 9.66735899e-01],
         [2.92977947e-03, 9.96612430e-01, 4.57778107e-04]],

        [[2.78023630e-02, 8.63673817e-03, 9.63560879e-01],
         [9.64354910e-03, 2.68554967e-03, 9.87670898e-01],
         [1.22986045e-02, 9.81567383e-01, 6.13403227e-03],
         [9.90447998e-01, 9.27734002e-03, 2.74657970e-04],
         [9.93530273e-01, 2.38037063e-03, 4.08936106e-03]],

        [[1.73650365e-02, 4.96841371e-02, 9.32950854e-01],
         [9.79705811e-01, 1.94701962e-02, 8.23974842e-04],
         [4.79125930e-03, 1.10778986e-02, 9.84130859e-01],
         [9.89319146e-01, 3.90613219e-03, 6.77469606e-03],
         [9.84344482e-01, 7.14111375e-03, 8.51440430e-03]],

        [[2.48717945e-02, 9.32830811e-01, 4.22973931e-02],
         [9.81903613e-01, 5.30989561e-03, 1.278650

In [36]:
memmap.flush()

#/mnt/f/backup/carlk4d/data/carlk/cachebio/genetics/onemil/id1000000.sid_1000000.seed0.byiid.bychrom/iid620000to630000.chrom1.bed
#F:\backup\carlk4d\data\carlk\cachebio\genetics\onemil\id1000000.sid_1000000.seed0.byiid.bychrom\iid620000to630000.chrom1.bed
209,629,000 bytes
#to 
#UKbio using 8 bits per probability, 
#Qctool convertion with 10K IID and 100K SNPs takes 3000 seconds
(py2) carlk@kadie2:/mnt/m/qctool$ build/release/qctool_v2.0.7 -g /mnt/f/backup/carlk4d/data/carlk/cachebio/genetics/onemil/id1000000.sid_1000000.seed0.byiid.bychrom/iid620000to630000.chrom1.bed -og /mnt/m/big.bgen
    
#-bgen-bits	For use when outputting BGEN files only. Tell QCTOOL to use this number of bits to store each probability.
#-bgen-compression	Specify what compression to use when outputting BGEN files only. This can be "none", "zlib", or "zstd".    

#https://bitbucket.org/gavinband/bgen/wiki/BGEN_in_the_UK_Biobank
8bit and zlib

(py2) carlk@kadie2:/mnt/m/qctool$ build/release/qctool_v2.0.7 -g /mnt/d/deldir/testsnps_1_10_50000_50000/data/chrom8.piece4of5.bed -og /mnt/m/1m.bgen -bg
en-bits 8 -bgen-compression zlib
9827000 bytes, 50Kiid, 805 snps, convertion takes 1.5 minutes
5492 bgen

In [12]:
from pysnptools.distreader.bgen import Bgen
bgenfile = r'M:\1m.bgen'
bgen = Bgen(bgenfile) #!!!cmk make relative
bgen.shape

cmk


(50000, 805)

In [14]:
memmapfile = r'm:/1m.dist.memmap'
memmap = DistMemMap.write(memmapfile,bgen.read(dtype=np.float16))

In [15]:
bgensize = os.stat(bgenfile).st_size
memmapsize = os.stat(memmapfile).st_size
bgensize,memmapsize,memmapsize/bgensize

(5623459, 242772418, 43.171368013886116)

In [24]:
bgen[1,50:60].read().val

array([[[nan, nan, nan],
        [ 0.,  0.,  1.],
        [ 0.,  0.,  1.],
        [ 0.,  0.,  1.],
        [ 0.,  0.,  1.],
        [nan, nan, nan],
        [ 0.,  0.,  1.],
        [nan, nan, nan],
        [ 0.,  0.,  1.],
        [ 0.,  0.,  1.]]])

In [29]:
from pysnptools.snpreader import Bed
bed = Bed(r'D:\deldir\testsnps_1_10_50000_50000\data\chrom8.piece4of5.bed',count_A1=True)
bed[0:20,300:5000].read().val

array([[nan,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0., nan],
       ...,
       [nan, nan,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0., nan, nan],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [24]:
iid_count = 500*1000
sid_count = 500
from pysnptools.distreader import DistGen
from pysnptools.distreader import Bgen
gen_file = r'm:\deldir\{0}x{1}.gen'.format(iid_count,sid_count)
sample_file2 = r'/mnt/m/deldir/{0}x{1}.sample'.format(iid_count,sid_count)
gen_file2 = r'/mnt/m/deldir/{0}x{1}.gen'.format(iid_count,sid_count)
bgen_file = r'm:\deldir\{0}x{1}.bgen'.format(iid_count,sid_count)
bgen_file2 = r'/mnt/m/deldir/{0}x{1}.bgen'.format(iid_count,sid_count)

In [15]:
distgen = DistGen(seed=332,iid_count=iid_count,sid_count=sid_count)
Bgen.genwrite(gen_file,distgen.read(),decimal_places=5)
print ('/mnt/m/qctool/build/release/qctool_v2.0.7 -g {0} -s {1} -og {2} -bgen-bits 8 -bgen-compression zlib'.format(gen_file2,sample_file2,bgen_file2))

/mnt/m/qctool/build/release/qctool_v2.0.7 -g /mnt/m/deldir/500000x500.gen -s /mnt/m/deldir/500000x500.sample -og /mnt/m/deldir/500000x500.bgen -bgen-bits 8 -bgen-compression zlib


In [18]:
distgen.pos

array([[1.0000000e+00, 0.0000000e+00, 1.0000000e+00],
       [1.0000000e+00, 0.0000000e+00, 6.1260010e+06],
       [1.0000000e+00, 0.0000000e+00, 1.2252001e+07],
       ...,
       [2.2000000e+01, 0.0000000e+00, 3.0498820e+09],
       [2.2000000e+01, 0.0000000e+00, 3.0560080e+09],
       [2.2000000e+01, 0.0000000e+00, 3.0621340e+09]])

In [16]:
bgen_file

'm:\\deldir\\500000x500.bgen'

In [25]:
bgen = Bgen(bgen_file,metadata=False) #!!!cmk metadata creation will fail, #!!!cmk False will create it?????
bgen.iid,bgen.sid,bgen.pos

cmk


(array([['', 'iid_0'],
        ['', 'iid_1'],
        ['', 'iid_2'],
        ...,
        ['', 'iid_499997'],
        ['', 'iid_499998'],
        ['', 'iid_499999']], dtype='<U10'),
 array(['sid_0', 'sid_1', 'sid_2', 'sid_3', 'sid_4', 'sid_5', 'sid_6',
        'sid_7', 'sid_8', 'sid_9', 'sid_10', 'sid_11', 'sid_12', 'sid_13',
        'sid_14', 'sid_15', 'sid_16', 'sid_17', 'sid_18', 'sid_19',
        'sid_20', 'sid_21', 'sid_22', 'sid_23', 'sid_24', 'sid_25',
        'sid_26', 'sid_27', 'sid_28', 'sid_29', 'sid_30', 'sid_31',
        'sid_32', 'sid_33', 'sid_34', 'sid_35', 'sid_36', 'sid_37',
        'sid_38', 'sid_39', 'sid_40', 'sid_41', 'sid_42', 'sid_43',
        'sid_44', 'sid_45', 'sid_46', 'sid_47', 'sid_48', 'sid_49',
        'sid_50', 'sid_51', 'sid_52', 'sid_53', 'sid_54', 'sid_55',
        'sid_56', 'sid_57', 'sid_58', 'sid_59', 'sid_60', 'sid_61',
        'sid_62', 'sid_63', 'sid_64', 'sid_65', 'sid_66', 'sid_67',
        'sid_68', 'sid_69', 'sid_70', 'sid_71', 'sid_72', 's

In [29]:
bgen[::100000,::50].read().val

array([[[0.46666667, 0.38823529, 0.14509804],
        [0.45098039, 0.24313725, 0.30588235],
        [0.44705882, 0.42745098, 0.1254902 ],
        [0.22352941, 0.40392157, 0.37254902],
        [       nan,        nan,        nan],
        [0.45098039, 0.41960784, 0.12941176],
        [0.34117647, 0.63137255, 0.02745098],
        [0.28235294, 0.38431373, 0.33333333],
        [0.25490196, 0.35686275, 0.38823529],
        [       nan,        nan,        nan]],

       [[0.76862745, 0.18823529, 0.04313725],
        [0.07843137, 0.48627451, 0.43529412],
        [0.5254902 , 0.36470588, 0.10980392],
        [0.75686275, 0.02745098, 0.21568627],
        [       nan,        nan,        nan],
        [0.35686275, 0.21960784, 0.42352941],
        [0.11372549, 0.4       , 0.48627451],
        [0.31372549, 0.35686275, 0.32941176],
        [0.37254902, 0.34117647, 0.28627451],
        [0.41960784, 0.4       , 0.18039216]],

       [[0.29803922, 0.41176471, 0.29019608],
        [0.17254902, 0.3450980

In [2]:
iid_count = 500*1000
sid_count = 5*1000*1000

from pysnptools.distreader import DistGen
from pysnptools.distreader import Bgen
distgen = DistGen(seed=332,iid_count=iid_count,sid_count=sid_count)
chrom_list = sorted(set(distgen.pos[:,0]))
len(chrom_list)

[1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 20.0,
 21.0,
 22.0]

In [5]:
import logging
logging.basicConfig(level=logging.INFO)
for chrom in chrom_list[::-1]:
    chromgen = distgen[:,distgen.pos[:,0]==chrom]
    print(chrom,chromgen.sid_count)
    name = '{0}x{1}.chrom{2}'.format(iid_count,sid_count,int(chrom))
    gen_file = r'm:\deldir\{0}.gen'.format(name)
    sample_file2 = r'/mnt/m/deldir/{0}.sample'.format(name)
    gen_file2 = r'/mnt/m/deldir/{0}.gen'.format(name)
    print("about to read {0}x{1}".format(chromgen.iid_count,chromgen.sid_count))
    Bgen.genwrite(gen_file,chromgen,decimal_places=5) #better in batches?
    print("done")
    bgen_file = r'm:\deldir\{0}.bgen'.format(name)
    bgen_file2 = r'/mnt/m/deldir/{0}.bgen'.format(name)
    print ('/mnt/m/qctool/build/release/qctool_v2.0.7 -g {0} -s {1} -og {2} -bgen-bits 8 -bgen-compression zlib'.format(gen_file2,sample_file2,bgen_file2))

22.0 91414
about to read 500000x91414


KeyboardInterrupt: 

In [6]:
chromgen

DistGen(seed=332,iid_count=100,sid_count=5000000,chrom_count=22,sid_batch_size=1000,cache_file=None)[:,[4908586,4908587,4908588,4908589,4908590,4908591,4908592,4908593,4908594,4908595,...]]

In [3]:
import logging
logging.basicConfig(level=logging.INFO)
from pysnptools.distreader import Bgen
bgen = Bgen(r'M:\deldir\500000x100.bgen',verbose=True)
print(bgen.shape)

Mapping variants: 100%|███████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 50045.39it/s]


cmk
(500000, 100)


In [3]:
%%time
bgen[0,50:60].read().val

TypeError: unhashable type: 'slice'

In [7]:
import logging
import os
logging.basicConfig(level=logging.INFO)
from pysnptools.distreader import DistMemMap
distmammap_file = r'M:\deldir\500000x100.dist.memmap'
if not os.path.exists(distmammap_file):
    distmemmap = DistMemMap.write(distmammap_file,bgen,dtype='float32',sid_batch_size=2)
else:
    distmemmap = DistMemMap(distmammap_file)
distmemmap    

DistMemMap('M:\deldir\500000x100.dist.memmap')

In [11]:
%%time
distmemmap.val[::50000,::10]

Wall time: 0 ns


memmap([[[       nan,        nan,        nan],
         [0.6784314 , 0.04313726, 0.2784314 ],
         [0.24705882, 0.29803923, 0.45490196],
         [0.4862745 , 0.08235294, 0.43137255],
         [0.28235295, 0.3882353 , 0.32941177],
         [       nan,        nan,        nan],
         [0.34509805, 0.18431373, 0.47058824],
         [0.34901962, 0.30588236, 0.34509805],
         [0.6666667 , 0.16862746, 0.16470589],
         [0.4117647 , 0.2627451 , 0.3254902 ]],

        [[0.39607844, 0.3137255 , 0.2901961 ],
         [0.30588236, 0.28627452, 0.40784314],
         [0.3647059 , 0.5019608 , 0.13333334],
         [0.24705882, 0.3882353 , 0.3647059 ],
         [       nan,        nan,        nan],
         [       nan,        nan,        nan],
         [0.44313726, 0.39215687, 0.16470589],
         [       nan,        nan,        nan],
         [       nan,        nan,        nan],
         [       nan,        nan,        nan]],

        [[       nan,        nan,        nan],
         

In [2]:
%%time
from pysnptools.distreader import Bgen
bgen2 = Bgen(r'M:\deldir\1x1000000.bgen',verbose=True) #!!!cmk why does this keep re-"Mapping variants" after metadata file already there?
#!!!cmk why slow even after mapping is done?
print(bgen2.shape)

Mapping variants: 100%|███████████████████████████████████████████████████| 1000000/1000000 [00:17<00:00, 58341.24it/s]


(1, 1000000)
Wall time: 1min 6s


In [4]:
%%time
bgen2.shape

Wall time: 0 ns


(1, 1000000)

In [5]:
%%time
start = 2500000
bgen2[-1,start:start+100].read().val

Wall time: 2 ms


array([], shape=(1, 0, 3), dtype=float64)

In [ ]:
import logging
import os
logging.basicConfig(level=logging.INFO)
from pysnptools.distreader import DistMemMap
distmammap_file2 = r'M:\deldir\1x5000000.dist.memmap'
if not os.path.exists(distmammap_file2):
    distmemmap2 = DistMemMap.write(distmammap_file2,bgen2,dtype='float32',sid_batch_size=10)
else:
    distmemmap2 = DistMemMap(distmammap_file2)
distmemmap2    

INFO:root:About to start allocating memmap 'M:\deldir\10x5000000b.dist.memmap'
INFO:root:Finished allocating memmap 'M:\deldir\10x5000000b.dist.memmap'. Size is 1040001460


In [1]:
%%time
#/mnt/m/qctool/build/release/qctool_v2.0.7 -g /mnt/m/deldir/1x1000000.gen -s /mnt/m/deldir/1x1000000.sample -og /mnt/m/deldir/1x1000000.bgen -bgen-bits 8 -bgen-compression zlib   
from bgen_reader import read_bgen
filename = r'm:\deldir\1x1000000.bgen'
bgen = read_bgen(filename,verbose=True)

Mapping variants: 100%|███████████████████████████████████████████████████| 1000000/1000000 [00:15<00:00, 66431.58it/s]


Wall time: 16.2 s


In [11]:
%%time
id_list = bgen['variants']['id']
len(id_list)

Wall time: 19.2 s


1000000

In [3]:
%%time
chrom_list = bgen['variants']['chrom']
len(chrom_list)

Wall time: 16.3 s


1000000

In [6]:
%%time
bgen["genotype"][500000].compute()['probs']

Wall time: 140 ms


array([[0.65490196, 0.2       , 0.14509804]])

In [8]:
%%time
del bgen
bgen = read_bgen(filename,verbose=True)

Mapping variants: 100%|███████████████████████████████████████████████████| 1000000/1000000 [00:14<00:00, 67933.42it/s]


Wall time: 15.4 s


In [9]:
%%time
chrom_list = bgen['variants']['chrom']
len(chrom_list)

Wall time: 18.5 s


1000000